In [4]:
 import pandas as pd

In [ ]:
train = pd.read_csv('/content/happywhale/csv/train.csv')

In [ ]:
# def rename_species(x):
#     if x == 'bottlenose_dolpin':
#         x = 'bottlenose_dolphin'
#     if x == 'kiler_whale':
#         x = 'killer_whale'
#     return x

# train['species'] = train['species'].apply(rename_species)